# Авиакомпания Россия

## Импорт данных и алгоритмы

Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

In [2]:
from russiya_airlines_solver import Solver
from russiya_airlines_interface import data_setting

Импорт исходных данных, который мы заслужили. При запуске ячейки должно открыться окно `tkinter` с интуитивным интерфейсом для импорта исходных данных. Данные о связках передаются в формате `XLS`. Данные о книгах передаются в формате `JSON`. Это **ВАЖНО!!!**

In [3]:
data, crew = data_setting()
print('Данные о книгах:\n', crew)
print('\nДанные о связках:\n')
data.head(5)

Данные о книгах:
           1       2       3       4       5       6
ВВЛ  125.54  122.84  129.97  127.26  123.32  130.64
МВЛ  119.88  117.30  124.10  121.51  117.76  124.75
СНГ  122.03  119.41  126.33  123.69  119.87  126.99

Данные о связках:



,День месяца,Время вылета,Связка,Назначение,Тип судна,Тип связи,Налет,Экипаж
0,1,02:55,route 1,city 61,А-320,ВВЛ,10:50,4
1,1,03:55,route 2,city 60,А-320,МВЛ,06:00,4
2,1,04:10,route 3,city 23,А-319,ВВЛ,03:00,3
3,1,04:40,route 4,city 32,А-319,МВЛ,08:35,3
4,1,05:00,route 5,city 8,А-319,ВВЛ,06:15,3


При запуске последовательно откроются два окна `tkinter`. В первом необходимо выбрать алгоритм и сортировку (рекомендуется выбирать A2, S1). Во втором задать веса критериев. По умолчанию все веса равны. Веса автоматически нормируются. 

In [4]:
sol = Solver(data, crew) 
print(sol.solve())

Время работы функции:	0.4794 сек

Решение записано в Solver.solutions;
Решение приведённое к формату Expert.xls записано в Solver.formated_solutions;
Спецификации решения сохранены в Solver.specifications;
Идентификатор решения: 1


Проверка распределения

In [5]:
identificator = 1
print('Значение целевой функции: {:.2f}'.format(Solver.solutions[identificator][0]))
for book in range(len(Solver.solutions[identificator][1])):
    print('Число связок на {}-ом рабочем столе: {}'.format(book, len(Solver.solutions[identificator][1][book])))

Значение целевой функции: 3.57
Число связок на 0-ом рабочем столе: 356
Число связок на 1-ом рабочем столе: 361
Число связок на 2-ом рабочем столе: 352
Число связок на 3-ом рабочем столе: 357
Число связок на 4-ом рабочем столе: 360
Число связок на 5-ом рабочем столе: 349


Экспорт решения

In [6]:
Solver.formated_solutions[identificator].to_excel('отчеты/solution.A2.S1.xls')

## Формирование отчёта

Импорт библиотек

In [7]:
from russiya_airlines_report import *

### Пример с найденным решением

Анализ решения

In [8]:
solution = pd.read_excel('solution.A2.S1.xls').iloc[1:] #Так надо делать с нашими данными из-за французской нумерации
solution = solution[solution.columns[1:3]].join(solution[solution.columns[4:]] + 1).fillna(value=0)
solution.head()

,Связка,Вылет,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
1,route 1,02:55,0,0,0,3,0,1,0,0,...,0,0,0,2,0,6,0,0,0,0
2,route 1,02:55,1,6,4,0,2,0,0,6,...,4,4,2,0,1,0,0,3,6,5
3,route 71,03:10,0,0,0,0,2,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,route 71,03:10,0,0,5,2,0,6,4,0,...,0,0,1,3,0,3,1,0,0,5
5,route 2,03:55,5,3,2,6,2,6,5,3,...,1,6,5,4,2,1,5,6,3,4


In [9]:
t1 = table_round_trip(solution, data)
t1.head()

,ВВЛ: БП,ВВЛ: Связки,МВЛ: БП,МВЛ: СВ,СНГ: БП,СНГ: Связки,Всего связок,Всего носных связок
0,815.0,251.0,295.0,84.0,71.0,21.0,356.0,64.0
1,825.0,256.0,287.0,82.0,80.0,23.0,361.0,62.0
2,769.0,236.0,322.0,92.0,83.0,24.0,352.0,66.0
3,799.0,246.0,306.0,87.0,86.0,24.0,357.0,64.0
4,820.0,254.0,300.0,86.0,70.0,20.0,360.0,60.0


In [10]:
t2 = table_flight_time(solution, data)
t2.head()

,ВВЛ: Планируемый налёт,ВВЛ: Планируемый ночной налёт,МВЛ: Планируемый налёт,МВЛ: Планируемый ночной налёт,СНГ: Планируемый налёт,СНГ: Планируемый ночной налёт
1,1273:40,403:32,556:55,38:03,158:50,73:51
2,1269:20,385:09,532:25,40:56,196:55,104:03
3,1229:20,395:32,598:55,40:38,202:50,102:39
4,1245:10,396:05,567:50,39:59,204:55,98:49
5,1243:50,380:50,577:20,55:22,156:00,72:40


In [11]:
t3 = table_flight_on_direction(solution, data)
t3.head()

,1,2,3,4,5,6,All,delta
city 14,14.0,15.0,14.0,12.0,10.0,18.0,83.0,8.0
city 21,5.0,6.0,5.0,4.0,7.0,4.0,31.0,3.0
city 7,12.0,13.0,8.0,10.0,6.0,13.0,62.0,7.0
city 54,1.0,2.0,1.0,1.0,1.0,2.0,8.0,1.0
city 33,5.0,5.0,5.0,6.0,4.0,6.0,31.0,2.0


In [12]:
t4 = table_flight_on_date(solution, data)
t4.head()

,1,2,3,4,5,6,All,delta
1,11.0,11.0,17.0,8.0,14.0,5.0,66.0,12.0
2,11.0,10.0,11.0,11.0,8.0,17.0,68.0,9.0
3,10.0,10.0,6.0,17.0,11.0,15.0,69.0,11.0
4,8.0,10.0,16.0,10.0,12.0,11.0,67.0,8.0
5,15.0,13.0,10.0,9.0,12.0,11.0,70.0,6.0


In [13]:
t5 = table_flight_on_plane(solution, data)
t5

,1,2,3,4,5,6,All,delta
А-319,246.0,254.0,235.0,237.0,250.0,232.0,1454.0,22.0
А-320,110.0,107.0,117.0,120.0,110.0,117.0,681.0,13.0


### Пример с решением эксперта

Анализ решения

In [14]:
solution = pd.read_excel('Expert.xls').iloc[1:].fillna(value=0)
solution.columns = list(map(str, solution.columns))
solution.head()

,Связка,Вылет,Тип,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
1,route 8,05:10,А 320,2,2,5,2,2,3,1,...,6,4,2,6,6,5,2,5,5,3
2,route 28,11:20,А 319,6,4,6,1,5,5,3,...,6,4,1,2,3,4,5,5,5,2
3,route 66,22:45,А 319,5,3,1,4,3,2,1,...,4,3,5,6,4,1,2,3,4,6
4,route 31,11:55,А 319,0,0,4,3,4,1,5,...,0,0,4,5,1,2,4,0,0,1
5,route 31,12:10,А 319,1,5,0,0,0,0,0,...,1,1,0,0,0,0,0,4,4,0


In [15]:
t1 = table_round_trip(solution, data)
t1.head()

,ВВЛ: БП,ВВЛ: Связки,МВЛ: БП,МВЛ: СВ,СНГ: БП,СНГ: Связки,Всего связок,Всего носных связок
0,805.0,246.0,306.0,88.0,86.0,24.0,358.0,59.0
1,789.0,243.0,317.0,90.0,75.0,22.0,355.0,66.0
2,803.0,249.0,293.0,85.0,74.0,22.0,356.0,62.0
3,789.0,245.0,307.0,88.0,77.0,21.0,354.0,68.0
4,798.0,245.0,304.0,86.0,74.0,22.0,353.0,62.0


In [16]:
t2 = table_flight_time(solution, data)
t2.head()

,ВВЛ: Планируемый налёт,ВВЛ: Планируемый ночной налёт,МВЛ: Планируемый налёт,МВЛ: Планируемый ночной налёт,СНГ: Планируемый налёт,СНГ: Планируемый ночной налёт
1,1283:00,382:02,574:15,42:12,197:50,92:31
2,1218:25,391:58,574:30,45:36,172:40,88:49
3,1241:35,358:20,546:25,38:13,177:40,89:35
4,1256:40,421:39,595:25,49:56,177:35,82:32
5,1224:40,380:23,586:10,42:00,177:20,90:00


In [17]:
t3 = table_flight_on_direction(solution, data)
t3.head()

,1,2,3,4,5,6,All,delta
city 14,15.0,13.0,14.0,14.0,13.0,14.0,83.0,2.0
city 21,4.0,8.0,0.0,6.0,5.0,8.0,31.0,8.0
city 7,11.0,8.0,13.0,12.0,9.0,9.0,62.0,5.0
city 54,0.0,2.0,2.0,0.0,2.0,2.0,8.0,2.0
city 33,8.0,7.0,5.0,1.0,7.0,3.0,31.0,7.0


In [18]:
t4 = table_flight_on_date(solution, data)
t4.head()

,1,2,3,4,5,6,All,delta
1,13.0,11.0,10.0,11.0,9.0,12.0,66.0,4.0
2,11.0,9.0,14.0,12.0,11.0,11.0,68.0,5.0
3,9.0,16.0,11.0,8.0,13.0,12.0,69.0,8.0
4,13.0,10.0,12.0,12.0,8.0,12.0,67.0,5.0
5,10.0,13.0,12.0,13.0,12.0,10.0,70.0,3.0


In [19]:
t5 = table_flight_on_plane(solution, data)
t5

,1,2,3,4,5,6,All,delta
А-319,236.0,239.0,254.0,245.0,237.0,243.0,1454.0,18.0
А-320,122.0,116.0,102.0,109.0,116.0,116.0,681.0,20.0


Конец файла